# Business Viability for an Italian Restaurant #
## Prompt ##
In a city of your choice, if someone is looking to open a restaurant, where would you recommend that 
they open it? Similarly, if a contractor is trying to start their own business, where would you recommend 
that they setup their office?

## Intro ## 
#### Problem ####
I am planning on investing in a new Italian restaurant somewhere in Vancouver, and before I invest I would like to know the viability of the business.

#### Data ####
Using ratings and number of restaurants in foursquare I will check the viability of an italian restraunt in the various neighborhoods of vancouver.  

## Library Imports and API Set Up ##

In [1]:
### Library Imports ###
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

import json
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
import folium # plotting library
from shapely.geometry import shape, Point

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from bs4 import BeautifulSoup
import requests
import re
import ast

In [2]:
### Four Square Credential Set Up ###
CLIENT_ID = 'FFL0XXXE3WUWP5KYUFZSRAUYPA0D3345OLUJZDWPXPUAVJXM' # your Foursquare ID
CLIENT_SECRET = 'FKP2AWR4KC4VXWM22SHZEGF3LYSG5SSDHPISP234TSRYIJLZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [3]:
### Venue Query Function: Returns Italian Venues within 1.65 km of lat lon position ###
 ## Vancouver area = 115km^2, #ofNeighborhoods = 22, so 22*1.664*pi = 115 ##
def queryAPI(latitude, longitude, search_query):
    radius = 2000 # meters
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
    results = requests.get(url).json()
    venues = results['response']['venues']
    return json_normalize(venues)

## Vancouver Neighborhood Location Set Up ##

In [4]:
### Load Vancouver GEO JSON data on neighborhoods ###
with open('vancouver.geojson') as f:
    js = json.load(f)

### Determine what neighborhood a latitude longitude point belongs to by using the GEO JSON data ###
def neighborhood(x):
    p = Point(x[1],x[0])
    # check each polygon to see if it contains the point
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(p):
            return feature['properties']['NAME']

### Find the center latitude and longitude points of a neighborhood in the GEO JSON data ###
def create_neighborhood_centers():
    # Updated Lists
    names = []
    longitude = []
    latitude = []
    
    # Go through geo data and determine center 
    for feature in js['features']:
        names.append(feature['properties']['NAME'])
        points = feature['geometry']['coordinates']
        x = [p[0] for p in points[0]]
        y = [p[1] for p in points[0]]
        longitude.append(sum(x) / len(points[0]))
        latitude.append(sum(y) / len(points[0]))
    
    # Create Dataframe
    neighborhood_centers = pd.DataFrame({'Neighborhoods':names,'latitude':latitude,'longitude':longitude})
    neighborhood_centers.to_csv("centers.csv",index=False)
    
# create_neighborhood_centers()

In [5]:
centers = pd.read_csv("centers.csv")

## Get Data ##

In [6]:
### Look for all venues of italian and sushi in each neighborhood and create and save a csv file of it ###  
# italian = []
# sushi = []

# for x in range(len(centers)):
#     print("Processing %d out of 21 Neighborhoods" % x)
#     print("Italian")
#     # Get data from API, Assign a neighborhood, Filter columns, and add dataframe to list for italian
#     venues = queryAPI(centers.iloc[x]['latitude'],centers.iloc[x]['longitude'], 'Italian')
#     venues['Neighborhood'] = centers.iloc[x]['Neighborhoods']
# #     venues = venues[['name', 'Neighborhood','categories','location.address','location.lat', 'location.lng','id']]
#     italian.append(venues)

#     print("Sushi")
#     # Get data from API, Assign a neighborhood, Filter columns, and add dataframe to list for sushi
#     venues = queryAPI(centers.iloc[x]['latitude'],centers.iloc[x]['longitude'], 'Sushi')
#     venues['Neighborhood'] = centers.iloc[x]['Neighborhoods']
# #     venues = venues[['name', 'Neighborhood','categories','location.address','location.lat', 'location.lng','id']]
#     sushi.append(venues)

# print("Deleting empty results")
# # Remove all neigborhoods that returned no results #
# italian = [x for x in italian if len(x)>0]
# sushi = [x for x in sushi if len(x)>0]

# # Put together the dataframes in italian and sushi 
# all_italian = italian[0]
# for x in range(1,len(italian)): 
#     all_italian = all_italian.append(italian[x])

# all_sushi = sushi[0]
# for x in range(1,len(sushi)): 
#     all_sushi = all_sushi.append(sushi[x])

In [7]:
def get_rating(x):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(x, CLIENT_ID, CLIENT_SECRET, VERSION)
 
    result = requests.get(url).json()

    try:
        return result['response']['venue']['rating']
    except:
        return -1

# # Get Ratings for Italian Restaurants
# all_italian['Rating'] = all_italian['id'].apply(get_rating)
# all_italian.to_csv('italian.csv',index=False)

# # Get Ratings for Sushi Restaurants
# all_sushi['Rating'] = all_sushi['id'].apply(get_rating)
# all_sushi.to_csv('sushi.csv',index=False)

In [8]:
# Scan in preprocessed Italian restaurant file
italian = pd.read_csv('italian.csv')
italian['Neighborhood'] = [neighborhood((italian.iloc[x]['location.lat'],italian.iloc[x]['location.lng'])) for x in range(len(italian))]
italian = italian[['name', 'Neighborhood','categories','location.address','location.lat', 'location.lng','id', 'Rating']]
italian = italian.drop_duplicates()

# Scan in preprocessed Sushi restaurant file
sushi = pd.read_csv('sushi.csv')
sushi['Neighborhood'] = [neighborhood((sushi.iloc[x]['location.lat'],sushi.iloc[x]['location.lng'])) for x in range(len(sushi))]
sushi = sushi[['name', 'Neighborhood','categories','location.address','location.lat', 'location.lng','id', 'Rating']]
sushi = sushi.drop_duplicates()

## Restaurants in the Area ###

In [9]:
dot_map = folium.Map(location = [49.2527,-123.1207], zoom_start=12)

# Add the Italian Restaurants as Green circle markers
group0 = folium.FeatureGroup(name='<span style=\\"color: green;\\">Italian</span>')
for lat, lng, label in zip(italian['location.lat'], italian['location.lng'],italian['name']):
    folium.Circle([lat,lng], 25, color='green', fill=True).add_child(folium.Popup(label)).add_to(group0)
group0.add_to(dot_map)

# Add the Sushi Restaurants as Red circle markers
group1 = folium.FeatureGroup(name='<span style=\\"color: red;\\">Sushi</span>')
for lat, lng, label in zip(sushi['location.lat'], sushi['location.lng'],sushi['name']):
    folium.Circle([lat,lng], 25, color='red', fill=True).add_child(folium.Popup(label)).add_to(group1)
group1.add_to(dot_map)

folium.map.LayerControl('topright', collapsed=False).add_to(dot_map)

dot_map

## Italian Restraunt Density ##

In [10]:
density = italian.groupby('Neighborhood')['id'].count().reset_index()

m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
van_geo = "vancouver.geojson"
m.choropleth(geo_data=van_geo,
    data=density,
    columns=['Neighborhood','id'],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name='Number of Italian Restaurants')

m

/anaconda3/lib/python3.6/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## Sushi Restaurant Density ##

In [11]:
density = sushi.groupby('Neighborhood')['id'].count().reset_index()

m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
van_geo = "vancouver.geojson"
m.choropleth(geo_data=van_geo,
    data=density,
    columns=['Neighborhood','id'],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name='Number of Sushi Restaurants')

m

## Italian Restaurant Ratings ##

In [12]:
italian = italian[italian['Rating'] >= 0]
m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
van_geo = "vancouver.geojson"
m.choropleth(geo_data=van_geo,
    data=italian,
    columns=['Neighborhood','Rating'],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name='Ratings')

m

## Top 5 Business Regions ## 

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=1650):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
# vancouver_venues = getNearbyVenues(names=centers['Neighborhoods'],
#                                    latitudes=centers['latitude'],
#                                    longitudes=centers['longitude']
#                                   )
# vancouver_venues.to_csv('vancouver_venues.csv', index=False)

In [15]:
vancouver_venues = pd.read_csv('vancouver_venues.csv')
temp = vancouver_venues.groupby(['Neighborhood','Venue Category'])['Venue'].count().reset_index().sort_values(['Neighborhood','Venue'], ascending=False)
temp = temp.groupby('Neighborhood').head(5).set_index('Neighborhood')

In [16]:
area = temp.iloc[0].name
print("--------TOP 5 Business Categories by Neighborhood--------")
print('--{}--'.format(area))
for x in range(len(temp)):
    if not area == temp.iloc[x].name:
        area = temp.iloc[x].name
        print('\n--{}--'.format(area))
    print('{}'.format(temp.iloc[x]['Venue Category']))

--------TOP 5 Business Categories by Neighborhood--------
--West Point Grey--
Beach
Coffee Shop
Park
Bakery
Café

--West End--
Ramen Restaurant
Bakery
Beach
Coffee Shop
Japanese Restaurant

--Victoria-Fraserview--
Burger Joint
Coffee Shop
Golf Course
Park
Sandwich Place

--Sunset--
Sandwich Place
Coffee Shop
Bakery
Restaurant
Bank

--Strathcona--
Brewery
Bakery
Park
Asian Restaurant
Auto Dealership

--South Cambie--
Garden
Farmers Market
Vegetarian / Vegan Restaurant
Vietnamese Restaurant
Bakery

--Shaughnessy--
Bakery
Garden
Park
Sandwich Place
Breakfast Spot

--Riley Park--
Garden
Vietnamese Restaurant
Farmers Market
Vegetarian / Vegan Restaurant
Bakery

--Renfrew-Collingwood--
Coffee Shop
Asian Restaurant
Candy Store
Indian Restaurant
Park

--Oakridge--
Garden
Sporting Goods Shop
Sushi Restaurant
Bank
Baseball Stadium

--Mount Pleasant--
Brewery
Coffee Shop
Bakery
Auto Dealership
Bookstore

--Marpole--
Café
Bubble Tea Shop
Chinese Restaurant
Hotel
Japanese Restaurant

--Kitsilano--


In [17]:
colors = ['red','blue','purple','green','black']
kmeans = KMeans(n_clusters=5, random_state=0).fit(vancouver_venues.iloc[:,4:6])

In [18]:
vancouver_venues['Colors'] = kmeans.labels_

dot_map = folium.Map(location = [49.2527,-123.1207], zoom_start=12)

for lat, lng, label, color in zip(vancouver_venues['Venue Latitude'], vancouver_venues['Venue Longitude'],vancouver_venues['Venue Category'], vancouver_venues['Colors']):
    folium.Circle([lat,lng], 25, color=colors[color], fill=True).add_child(folium.Popup(label)).add_to(dot_map)
    
dot_map

In [19]:
from scipy.spatial.distance import pdist, squareform
from haversine import haversine
distance_matrix = squareform(pdist(vancouver_venues.iloc[:,4:6], (lambda u,v: haversine(u,v))))
db = DBSCAN(eps=.225, min_samples=5, metric="precomputed")
y_db = db.fit_predict(distance_matrix)

In [20]:
y_db

array([-1, -1,  0, -1, -1,  1,  0, -1, -1,  0, -1, -1,  0, -1,  0, -1,  1,
       -1, -1, -1, -1, -1, -1,  0, -1,  0, -1,  0,  0,  1,  2, -1,  3,  2,
        2,  3,  2,  2,  3,  3,  2,  3,  3,  3,  3,  3,  2,  2,  3, -1, -1,
       -1,  3,  2, -1, -1,  2,  2,  3, -1,  4,  4,  4,  4,  5,  5,  4,  5,
        6,  7,  5,  8,  6, -1, -1,  8,  7,  8,  6,  8,  9,  9,  8,  8,  8,
        5, -1,  8,  9, -1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 13, 13, 12, 11, 11, 13, 11, 11, 11, 12, 11, 12, 12, 11, 11, 11,
       11, 11, 11, 13, 11, 13, 11, 11, 11, 13, 12, 11, 12, 12, 14, 14, -1,
       -1, -1, 15, 14, 14, 15, 15, 15, -1, 15, 15, -1, 15, 15, 15, 15, 14,
       14, 15, 14, 14, 14, -1, 15, 15, 15, 15, -1, -1, 16, -1, 16, 16, 17,
       -1, -1, 16, 18, 17, 16, -1, 16, 18, 18, 18, 18, 16, -1, 16, -1, -1,
       16, 18, 17, 17, 16, 16, -1, -1, 19, 19, 19, 19, -1, -1, -1, 19, 19,
       19, 19, 19,  7, 19

In [21]:
colors = ['red','blue','purple','green','black','yellow', 'brown','white','cyan']
vancouver_venues['Colors'] = y_db

dot_map = folium.Map(location = [49.2527,-123.1207], zoom_start=12)

for lat, lng, label, color in zip(vancouver_venues['Venue Latitude'], vancouver_venues['Venue Longitude'],vancouver_venues['Venue Category'], vancouver_venues['Colors']):
    if color < 0: 
        folium.Circle([lat,lng], 25, color='gray', fill=True).add_child(folium.Popup(label)).add_to(dot_map)
    else:
        folium.Circle([lat,lng], 25, color=colors[int((color-1)/4)], fill=True).add_child(folium.Popup(label)).add_to(dot_map)
    
dot_map

## Rental Properties from https://www.spacelist.ca/listings/bc/vancouver ##

In [22]:
def get_commercial_rent():
    # Set up the link, get the html from the page, and put the contents into beautiful soup
    page_link = 'https://www.spacelist.ca/listings/bc/vancouver'
    page_response = requests.get(page_link, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")

    # Seperate out the listngs to get id and rent
    listings = page_content.find_all('a', {'class':'listing-card'})

    # Get the location data out of the html and turn it into a dictionary
    locations = ast.literal_eval(re.findall("data-data='{\"type\":\"FeatureCollection\",\"features\":\[(.*)\]\}'", str(page_content.find('div', id='cluster-map')))[0])

    # Set a list to store values to and grab the id, latitude, and longitude
    location_list = []
    for i,x in enumerate(locations):
        location_id = locations[i]['properties']['id']
        longitude = locations[i]['geometry']['coordinates'][0]
        latitude = locations[i]['geometry']['coordinates'][1]
        location_list.append([location_id, latitude, longitude])

    # Convert location_list into a pandas dataframe and export to a csv
    l = pd.DataFrame(location_list, columns=['id', 'lat', 'lon'])
    l.to_csv('locations.csv', index=False)

    # Collect A Single Page of rent data that we got from the above url and add it to a list called rent 
    rent = []
    for listing in listings:
        listing_id = re.findall('data-listing=\"(.*\d)\" ',str(listing))[0]
        try:
            listing_price = listing.find('div', {'class':'rent_per_month'}).text
        except:
            listing_price = None

        rent.append([listing_id, listing_price])

    # Collect all the pages of Data from page 2 - 47, then extract the rent and id and place it on a list called rent
    for x in range(2,int(924/20)+2):
        page_link = 'https://www.spacelist.ca/listings/bc/vancouver/page/{}'.format(x)
        page_response = requests.get(page_link, timeout=5)
        page_content = BeautifulSoup(page_response.content, "html.parser")
        listings = page_content.find_all('a', {'class':'listing-card'})

        for listing in listings:
            listing_id = re.findall('data-listing=\"(.*\d)\" ',str(listing))[0]
            try:
                listing_price = listing.find('div', {'class':'rent_per_month'}).text
            except:
                listing_price = None

            rent.append([listing_id, listing_price])

    # Convert rent into a pandas dataframe and export to a csv
    r = pd.DataFrame(rent, columns=['id', 'rent'])
    r.to_csv('rent.csv',index=False)
    
#get_commercial_rent()

In [23]:
locations = pd.read_csv('locations.csv') 
rent = pd.read_csv('rent.csv') 
locations['neighborhood'] = [neighborhood((locations.iloc[x]['lat'],locations.iloc[x]['lon'])) for x in range(len(locations))]
rentals = pd.merge(rent, locations, on=['id','id'])

In [24]:
def to_numeric(x):
        x = x.replace('$','')
        x = x.replace('/mo','')
        x = x.replace(',','')
        
        if x == 'Contact' or x == 'None':
            return None
        
        if '-' in x: 
            values = x.split('-')
            return (float(values[1]) + float(values[0]))/2
        else:
            return float(x)

rentals['rent'] = rentals['rent'].astype(str).apply(to_numeric)

## Number of Available Rental Properties by Neighborhood ##

In [25]:
density = rentals.groupby('neighborhood')['id'].count().reset_index()

m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
van_geo = "vancouver.geojson"
m.choropleth(geo_data=van_geo,
    data=density,
    columns=['neighborhood','id'],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name='Properties for Rent')

m

/anaconda3/lib/python3.6/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [26]:
density

,neighborhood,id
0,Arbutus-Ridge,1
1,Downtown,405
2,Dunbar-Southlands,2
3,Fairview,87
4,Grandview-Woodland,23
5,Hastings-Sunrise,23
6,Kensington-Cedar Cottage,17
7,Kerrisdale,8
8,Killarney,8
9,Kitsilano,40


## Rental Property Locations ## 

In [27]:
m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
for lat, lng in zip(rentals['lat'], rentals['lon']):
    folium.Circle([lat,lng], 25, color='green', fill=True).add_to(m)
m

## Median Rental Price by Neighborhood ##

In [28]:
density = rentals[rentals['rent']<100000].groupby('neighborhood')['rent'].median().reset_index()
 
m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
van_geo = "vancouver.geojson"
m.choropleth(geo_data=van_geo,
    data=density,
    columns=['neighborhood','rent'],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.5, 
    line_opacity=0.9,
    legend_name='Rental Price')

m

In [29]:
density

,neighborhood,rent
0,Arbutus-Ridge,3947.0
1,Downtown,6164.0
2,Dunbar-Southlands,7524.0
3,Fairview,4695.0
4,Grandview-Woodland,12942.0
5,Hastings-Sunrise,5245.0
6,Kensington-Cedar Cottage,5222.0
7,Kerrisdale,7233.0
8,Killarney,5025.5
9,Kitsilano,4684.0


## Rental Properties with Listed Price ##

In [30]:
r = rentals.dropna()
m = folium.Map(location = [49.2527,-123.1207], zoom_start=12)
for lat, lng, label in zip(r['lat'], r['lon'], r['rent']):
    folium.Circle([lat,lng], 25, color='green', fill=True).add_child(folium.Popup(label)).add_to(m)
m